In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine, text
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score


postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'heartdisease'
engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))
data = pd.read_sql_query('select * from heartdisease',con=engine)
# No need for an open connection, 
# because you're only doing a single query
engine.dispose()

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       303 non-null    float64
 1   1       303 non-null    float64
 2   2       303 non-null    float64
 3   3       303 non-null    float64
 4   4       303 non-null    float64
 5   5       303 non-null    float64
 6   6       303 non-null    float64
 7   7       303 non-null    float64
 8   8       303 non-null    float64
 9   9       303 non-null    float64
 10  10      303 non-null    float64
 11  11      303 non-null    object 
 12  12      303 non-null    object 
 13  13      303 non-null    int64  
dtypes: float64(11), int64(1), object(2)
memory usage: 33.3+ KB


In [3]:
data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,63.0,1.0,1.0,145.0,233.0,1.0,2.0,150.0,0.0,2.3,3.0,0.0,6.0,0
1,67.0,1.0,4.0,160.0,286.0,0.0,2.0,108.0,1.0,1.5,2.0,3.0,3.0,2
2,67.0,1.0,4.0,120.0,229.0,0.0,2.0,129.0,1.0,2.6,2.0,2.0,7.0,1
3,37.0,1.0,3.0,130.0,250.0,0.0,0.0,187.0,0.0,3.5,3.0,0.0,3.0,0
4,41.0,0.0,2.0,130.0,204.0,0.0,2.0,172.0,0.0,1.4,1.0,0.0,3.0,0


In [6]:
# Define the features and the outcome
X = data.iloc[:, :13]
y = data.iloc[:, 13]

# Replace missing values (marked by `?`) with a `0`
X = X.replace(to_replace='?', value=0)

# Binarize `y` so that `1` means heart disease diagnosis and `0` means no diagnosis
y = np.where(y > 0, 1, 0)

In [9]:
# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [10]:
# Apply k-means clustering with 2 clusters
kmeans = KMeans(n_clusters=2, random_state=42)
kmeans.fit(X_scaled)

# Get the cluster labels
cluster_labels = kmeans.labels_


C:\Users\sekha\Anaconda3_1\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [12]:
# Create a confusion matrix to understand the mapping
conf_matrix = confusion_matrix(y, cluster_labels)
print("Confusion Matrix:")
print(conf_matrix)

# Determine the majority label for each cluster
majority_label_map = {}
for cluster in range(2):
 # Use Pandas Series to compute mode
    majority_label = pd.Series(y[cluster_labels == cluster]).mode()[0]
    majority_label_map[cluster] = majority_label

# Map the cluster labels to the majority labels
mapped_labels = np.vectorize(majority_label_map.get)(cluster_labels)

# Evaluate the clustering performance
accuracy = accuracy_score(y, mapped_labels)
classification_rep = classification_report(y, mapped_labels)

print(f"Accuracy: {accuracy}")
print("Classification Report:")
print(classification_rep)

Confusion Matrix:
[[ 13 151]
 [101  38]]
Accuracy: 0.8316831683168316
Classification Report:
              precision    recall  f1-score   support

           0       0.80      0.92      0.86       164
           1       0.89      0.73      0.80       139

    accuracy                           0.83       303
   macro avg       0.84      0.82      0.83       303
weighted avg       0.84      0.83      0.83       303

